In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn import model_selection
import matplotlib.pyplot as plt
import torch
import scipy.stats as stats
import torch.nn as nn

In [2]:
# Lets grab our data from the pca transformation
X_numpy = np.load('pca_transformed_data_v1.npy', allow_pickle=True)
y_numpy = np.load('label.npy', allow_pickle=True)

In [3]:
X = torch.from_numpy(X_numpy.astype(np.float32)).squeeze()
y = torch.from_numpy(y_numpy.astype(np.float32))

In [4]:
#lets split our data
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size=0.2,
                                                                           random_state=50)

In [5]:
#send the data into gpu
device = 'mps' if torch.backends.mps.is_available() else 'cpu'
X_train,X_test, y_train,y_test = X_train.to(device), X_test.to(device), y_train.to(device), y_test.to(device)

In [8]:
# lets create a model for hyperparameter tuning
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.layers = nn.ModuleList()
        self.layers.append(nn.Linear(18, 28))
        self.layers.append(nn.ReLU())
        self.layers.append(nn.Linear(28, 1))
        
        #add forward function
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

In [9]:
from pathlib import Path
loaded_model_1 = NeuralNetwork()
MODEL_PATH = Path('models')
MODEL_NAME = "Neural_2.pth"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME
loaded_model_1.load_state_dict(torch.load(f=MODEL_SAVE_PATH))

<All keys matched successfully>

In [10]:
loaded_model_1.to(device)

NeuralNetwork(
  (layers): ModuleList(
    (0): Linear(in_features=18, out_features=28, bias=True)
    (1): ReLU()
    (2): Linear(in_features=28, out_features=1, bias=True)
  )
)

In [13]:
# Score
with torch.inference_mode():
    test_pred = loaded_model_1(X_test).to('cpu')
    score = stats.spearmanr(y_test.to('cpu'), test_pred)
print(f'The spearman correlation coefficient for neural network model is: {score.statistic}')

The spearman correlation coefficient for neural network model is: 0.3540954043688203
